In [11]:
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

from tqdm import tqdm
from fuzzywuzzy import fuzz
import random
import re

In [12]:
#Leemos nuestro csv con los datos totales de peliculas y cortos extaidos de la api MovieDatabase 
lista_api=pd.read_csv('datos_totales.csv')
lista_api.drop(columns= ['Unnamed: 0'], inplace= True)
lista_api.head()

In [ ]:
#Hacemos un random para la muestra

lista_tuplas = [tuple(x) for x in lista_api.values]
lista_tuplas= random.sample(lista_tuplas, 10000)


In [13]:
def extraccion_caracteristicas_peliculas(url): 
    
    driver = webdriver.Chrome()
    driver.get(url)
    sleep(2)
    driver.maximize_window()

    lista_tupla_imdb=[]

    for tupla in tqdm(lista_tuplas):

        lista_imdb =[]
        
        driver.find_element("css selector", "#suggestion-search").send_keys(tupla[-1], Keys.ENTER)

            
        id_pelicula=tupla[-1]    
        lista_imdb.append(id_pelicula)
            
        sleep(1)
            
        nombre_pelicula=tupla[1]
        lista_imdb.append(nombre_pelicula)
            
        tipo=tupla[0]
        lista_imdb.append(tipo)
            
        sleep(1)
        try: 
                
                try: 
                    director= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[1]/div').text
                    director_correcto = re.sub(r'([a-z])([A-Z])', r'\1 \2', director)   
                    lista_imdb.append(director_correcto) 
                except: 
                    director=driver.find_element('xpath', '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div/div[1]/section/div[2]/div/ul/li[1]/div').text
                    director_correcto = re.sub(r'([a-z])([A-Z])', r'\1 \2', director)   
                    lista_imdb.append(director_correcto)
            
        except: 
                
            lista_imdb.append("No especificado")
            
        sleep(2)
            
        try: 
                try: 
                    guionista= driver.find_element('xpath','//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[2]/div').text
                    guionista_correcto = re.sub(r'([a-z])([A-Z])', r'\1 \2', guionista)
                    lista_imdb.append(guionista_correcto)
                except: 
                    guionista=driver.find_element('xpath', '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div/div[1]/section/div[2]/div/ul/li[2]/div').text
                    guionista_correcto = re.sub(r'([a-z])([A-Z])', r'\1 \2', guionista)
                    lista_imdb.append(guionista_correcto)
                
            
        except: 
            lista_imdb.append("No especificado")
            
        sleep(2)
            
        try: 
                for i in range(2, 4):
                    duracion= driver.find_element("xpath",f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/ul/li[{i}]').text
                    if 'min' in duracion or 'h' in duracion: 
                        lista_imdb.append(duracion)
                        break
                    else:
                        pass           
        except:
            duracion =("No especificado")
            lista_imdb.append(duracion)
                                
                            
        sleep(3)
            
        try: 
            puntuacion = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[2]/div/div[1]/a/span/div/div[2]/div[1]/span[1]').text
            
            lista_imdb.append(puntuacion.replace(",", "."))
                
        except: 
            lista_imdb.append("No especificado")

        sleep(1)
            
        try:  
                
            #print("el argumento") 
            argumento = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/p/span[3]').text
                                                        
            lista_imdb.append(argumento)
            
        except:
            #print("sin argumento")
            lista_imdb.append("No especificado")
            
        sleep(2)
                
        lista_tupla_imdb.append(tuple(lista_imdb))

        sleep(4)
        driver.back()
    
    return lista_tupla_imdb
        

In [ ]:
resultados= extraccion_caracteristicas_peliculas("https://www.imdb.com/")

In [ ]:
caracteristicas_peliculas=pd.DataFrame(resultados, columns=["Id", "Nombre","Tipo", "Director", "Guionista", "Duracion", "Puntuacion", "Argumento"])

caracteristicas_peliculas.head()
caracteristicas_peliculas.drop_duplicates()